# **Heart Disease Veri Setinin Ön İşleme Adımları ve Naive Bayes Sınıflandırılması**

**Bu veri setini ön işleme adımlarından geçirdikten sonra Naive bayes algortimasıyla sınıflandırdıktan sonra çeşitli parametrelerle accuracy(doğruluk oranı) nasıl değiştiğini gözlemleyeceğiz.**

**VERİ SETİ HAKKINDA BİLGİLER**

*Kalp hastalığı için oluşturulan veri setinde 500 sample(observation, örnek,satır) 6 feature(attributes,etiket,özellik,label,sütun) bulunmaktadır. Veri seti incelendiğinde son sütunun tahmin etmek istediğimiz özellik olduğunu görebiliriz. Son sütun yani HeartDisease; target(hedef) değişkenimizdir.  

Yapılması gereken;
* Eksik veri olup olmadığını kontrol etmek(varsa tamalamak veya çıkartmak),
* Tekrarlayan örnekler varsa çıkartmak,
* Sınıf dağılımını kontrol ederek veri setinin dengesiz olup olmadığını tespit etmek(dengesizlik varsa SMOTE veya çeşitli örnek artırma-azaltma metodlarını denemek)
* Veri setindeki sütunların istatiksel değerlerini kontrol etmek(aykırı veri olup olmadığını görmek için)


In [13]:
#Veri ön işleme için kullanacağımız kütüphaneler ve veri setinin yüklenmesi
import pandas as pd
import numpy as np

df=pd.read_csv("/kaggle/input/heart-prediction-dataset-quantum/Heart Prediction Quantum Dataset.csv")
df

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature,HeartDisease
0,68,1,105,191,107,8.362241,1
1,58,0,97,249,89,9.249002,0
2,44,0,93,190,82,7.942542,1
3,72,1,93,183,101,6.495155,1
4,37,0,145,166,103,7.653900,1
...,...,...,...,...,...,...,...
495,34,0,126,292,116,9.303403,0
496,41,0,164,248,114,9.067889,0
497,45,1,159,175,75,8.718708,0
498,55,0,107,157,101,7.337650,1


In [14]:
#Eksik verinin olup olmadığının kontrol edilmesi
df.isnull().sum()

Age                      0
Gender                   0
BloodPressure            0
Cholesterol              0
HeartRate                0
QuantumPatternFeature    0
HeartDisease             0
dtype: int64

Bu çıktıya göre veri setinde herhangi bir eksik veri yok.

In [20]:
df.count()

Age                      500
Gender                   500
BloodPressure            500
Cholesterol              500
HeartRate                500
QuantumPatternFeature    500
HeartDisease             500
dtype: int64

In [15]:
#Tekrarlayan satırlar kaldırılarak inplace parametresiyle veri setini güncelledik
df.drop_duplicates(inplace=True)

Tekrarlanan satırlar temizlendi

In [19]:
#df'i tekrar sayarak herhangi bir tekrarlayan veri olmadığını öğreniyoruz.
df.count()

Age                      500
Gender                   500
BloodPressure            500
Cholesterol              500
HeartRate                500
QuantumPatternFeature    500
HeartDisease             500
dtype: int64

In [21]:
#Verilerin nasıl değiştiğini kontrol ediyoruz
df.describe()

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature,HeartDisease
count,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,54.864000,0.468000,132.874000,221.50000,88.766000,8.317407,0.600000
std,14.315004,0.499475,26.418516,43.86363,17.417289,0.919629,0.490389
min,30.000000,0.000000,90.000000,150.00000,60.000000,6.164692,0.000000
25%,43.000000,0.000000,111.000000,183.75000,73.000000,7.675779,0.000000
50%,55.000000,0.000000,132.000000,221.00000,89.000000,8.323064,1.000000
75%,66.250000,1.000000,155.000000,258.00000,104.000000,8.935999,1.000000
max,79.000000,1.000000,179.000000,299.00000,119.000000,10.784886,1.000000


Veri setindeki verilerin sayısal mı yoksa kategorik mi olduğunu tespit etmek için kolay bir yöntem. Eğer ilgili sütunda istatiksel veriler mevcutsa sayısal verilerden bahsettiğimiz çıkarımı yapılabilir.

In [22]:
#Tahmin etmek istediğimiz sütunu bir değişkene atıyoruz
target=df["HeartDisease"]
target

0      1
1      0
2      1
3      1
4      1
      ..
495    0
496    0
497    0
498    1
499    0
Name: HeartDisease, Length: 500, dtype: int64

Sınıf veri setinden ayrıldı.

In [23]:
#Sınıfları içeren sütun hariç geri kalanı başka bir veri setinde tutuyoruz.
df_final=df.drop(columns="HeartDisease", axis=0)
df_final

,Age,Gender,BloodPressure,Cholesterol,HeartRate,QuantumPatternFeature
0,68,1,105,191,107,8.362241
1,58,0,97,249,89,9.249002
2,44,0,93,190,82,7.942542
3,72,1,93,183,101,6.495155
4,37,0,145,166,103,7.653900
...,...,...,...,...,...,...
495,34,0,126,292,116,9.303403
496,41,0,164,248,114,9.067889
497,45,1,159,175,75,8.718708
498,55,0,107,157,101,7.337650


Tahmin edilecek sütun ve tahmin edilmesi istenenler için farklı veri setleri oluşturuldu.

Veri setinde dengesizlik olup olmadığını kontrol ediyoruz. Çünkü dengesizlik modelin belirli bir 
sınıfı daha iyi öğrendiği anlamına gelir. Aşağıdaki yorum satırını kaldırarak diğer sınıftan kaç tane olduğunu görebilirsiniz. 0=>200 tane 1=>300 tane var. Dengesizlik var. Bunu önlemek için SMOTE kullanacağız.

In [24]:
#df[df["HeartDisease"]==1].count()
df[df["HeartDisease"]==0].count()

Age                      200
Gender                   200
BloodPressure            200
Cholesterol              200
HeartRate                200
QuantumPatternFeature    200
HeartDisease             200
dtype: int64

Hasta olan 200 kişi var. Hasta olmayan kişi sayısı ise 300. Durumda ya veri 300 örneklik grubu 200' e indirmeyi ya da 200 örneklk sınıfı 300 örneğe çıkartmayı seçeceğiz. Veri setinin boyutunu göz önüne alarak SMOTE ile veri artırmayı tercih ettim.


**Model Oluşturulması ve Eğitim Süreci**

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

#Veri setini eğitim(%80) ve test(%20) verisi olarak ayır 
X_test, X_train, y_test, y_train = train_test_split(df, target, test_size = 0.2, random_state=99)

smote=SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)

In [30]:
#Veri artırmanın sonuçlanıp sonuçlanmadığına bakıyoruz
from collections import Counter
print(Counter(y_train_resampled))

Counter({1: 59, 0: 59})


In [31]:
# Naive Bayes modelini başlat
naive_bayes = GaussianNB()

# Modeli eğit
naive_bayes.fit(X_train_resampled, y_train_resampled)

# Test verisiyle tahmin yap
y_pred = naive_bayes.predict(X_test)

# Doğruluk oranını hesapla
acc = accuracy_score(y_pred, y_test)
print(f"Accuracy: {acc:.4f}")

Accuracy: 1.0000


**Farklı Parametreler İle Değişim**

In [32]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_resampled, y_train_resampled)

y_pred_mnb = mnb.predict(X_test)
acc_mnb = accuracy_score(y_test, y_pred_mnb)
print(f"Accuracy: {acc_mnb:.4f}")

Accuracy: 0.8125


In [33]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train, y_train)
y_pred_bnb = bnb.predict(X_test)
acc_bnb = accuracy_score(y_test, y_pred_bnb)

print(f"Accuracy: {acc_mnb:.4f}")

Accuracy: 0.8125


Değiştirilen parametreler olarak Naive bayes türleri karşılaştırıldı. Siz isterseniz kullanmak istediğini türü yazdıktan sonra soru işareti kullanarak hangi parametreleri kullanabileceğinize göre çeşitli denemeler yapabilirsiniz. 

In [34]:
GaussianNB?

Init signature: GaussianNB(*, priors=None, var_smoothing=1e-09)
Docstring:     
Gaussian Naive Bayes (GaussianNB).

Can perform online updates to model parameters via :meth:`partial_fit`.
For details on algorithm used to update feature means and variance online,
see Stanford CS tech report STAN-CS-79-773 by Chan, Golub, and LeVeque:

    http://i.stanford.edu/pub/cstr/reports/cs/tr/79/773/CS-TR-79-773.pdf

Read more in the :ref:`User Guide <gaussian_naive_bayes>`.

Parameters
----------
priors : array-like of shape (n_classes,), default=None
    Prior probabilities of the classes. If specified, the priors are not
    adjusted according to the data.

var_smoothing : float, default=1e-9
    Portion of the largest variance of all features that is added to
    variances for calculation stability.

    .. versionadded:: 0.20

Attributes
----------
class_count_ : ndarray of shape (n_classes,)
    number of training samples observed in each class.

class_prior_ : ndarray of shape (n_classes,)